In [5]:
from scripts.bugsinpy_utils import *
from scripts.embedding import BATCH_SIZE, model
from run_ast import extract_chunks
import numpy
import os, json

/home/pooria/bootcamp/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Extract the code chunks for each bug based on their buggy_commit_id

In [3]:
projects = get_projects()
for project in projects:
    clone_project(project)
    bugs = get_bugs(project)
    for bug in bugs:
        info = get_bug_info(project, bug)
        checkout_to_commit(project, info["buggy_commit_id"])
        os.makedirs(f"dataset/{project}/{bug}", exist_ok=True)
        chunks = extract_chunks(f"tmp/{project}")
        with open(
            f"dataset/{project}/{bug}/code_chunks.json", "w", encoding="utf-8"
        ) as f:
            json.dump(chunks, f, indent=2)

Previous HEAD position was c404d1f716 Merge pull request #17587 from QuLogic/fix-docs
HEAD is now at cdf9e30e4f Merge pull request #14610 from meeseeksmachine/auto-backport-of-pr-14579-on-v3.1.x
Processing files: 100%|██████████| 768/768 [00:15<00:00, 51.14file/s]
Previous HEAD position was cdf9e30e4f Merge pull request #14610 from meeseeksmachine/auto-backport-of-pr-14579-on-v3.1.x
HEAD is now at dbc35a9d62 Merge pull request #16683 from anntzer/gluespec
Processing files: 100%|██████████| 761/761 [00:13<00:00, 55.63file/s]
Previous HEAD position was dbc35a9d62 Merge pull request #16683 from anntzer/gluespec
HEAD is now at e92685a264 Merge pull request #15254 from anntzer/propagate-decorators
Processing files: 100%|██████████| 754/754 [00:14<00:00, 51.23file/s]
Previous HEAD position was e92685a264 Merge pull request #15254 from anntzer/propagate-decorators
HEAD is now at fb9e72309a Merge pull request #17462 from anntzer/gtk_render_figure
Processing files: 100%|██████████| 759/759 [00:

In [6]:
for project in projects:
    bugs = get_bugs(project)
    for bug in bugs:
        with open(f"dataset/{project}/{bug}/code_chunks.json", "r", encoding="utf-8") as f:
            chunks = json.loads(f.read())
            # Embedding each text
            texts = [chunk["code"] for chunk in chunks]
            if BATCH_SIZE:
                embeddings = model.encode(
                    texts, batch_size=BATCH_SIZE, show_progress_bar=True
                )
            else:
                embeddings = model.encode(texts, show_progress_bar=True)
            
            numpy.save(f"dataset/{project}/{bug}/embedding.npy", embeddings)
                

Batches:   0%|          | 0/70 [00:00<?, ?it/s]